## 1. Import libraries

In [ ]:
import numpy
from matplotlib import pyplot
from matplotlib.ticker import FixedLocator, FixedFormatter

import matplotlib.pylab
params = {'legend.fontsize': 8,
         'font.family':'sans-serif',
         'font.sans-serif':'Times New Roman',
         'font.style':'normal',
         'font.weight':'normal',
         'axes.labelsize': 'small', #'medium', #'xx-large',
         'axes.titlesize':'medium',
         'axes.spines.top': False,
         'axes.spines.right': False,
         'axes.linewidth': 1.0,
         'xtick.labelsize': 6, #small #medium
         'ytick.labelsize': 6,
         'mathtext.fontset':'cm',
         'text.latex.preamble': r'usepackage{amsmath}',}

matplotlib.pylab.rcParams.update(params)
saved_img_path = './saved_images'

## 2. GLM Regression

#### Visualization of the GLM Poisson

In [ ]:
def get_regression_value(x, w, mu):
    return numpy.exp(numpy.dot(x, w) + mu)

def get_xlimit_ylimit(X=None, y=None, Xc=None, Mat_dists=None, V=None, v_ext=None, B=None, b_ext=None, BETA=None):
    if (Xc is None) and (Mat_dists is None) and (V is None) and (B is None) and (BETA is None) and (v_ext is not None) and (b_ext is not None):
        y_pred = get_regression_value(X, v_ext, b_ext)
        xmin, xmax = min(X), max(abs(X))
        ymin, ymax = 0, max(max(y), max(y_pred))
        
        return xmin, xmax, ymin, ymax

    else:
        # for V = v_1 to v_n
        y_pred_beta = []
        for i in range(BETA.size):
            xpoints_beta, y_beta = X[Mat_dists[i] <= BETA[i]], y[Mat_dists[i] <= BETA[i]]
            if xpoints_beta.size != 0:
                y_pred_beta = numpy.concatenate((y_pred_beta, get_regression_value(xpoints_beta, V[i], B[i])))
            
        # for v_ext = w_ext (out of beta region)
        mask = Mat_dists > BETA.reshape(BETA.size,1)
        mask = numpy.prod(mask, axis=0).astype(bool)
        xpoints_out_beta, y_out_beta = X[mask], y[mask]
        if xpoints_out_beta.size != 0:
            y_pred_out_beta = get_regression_value(xpoints_out_beta, v_ext, b_ext)
            
        X = X.reshape(X.shape[0])
        #xmax = numpy.round(numpy.ceil(max(max(abs(X)), abs(Xc[numpy.argmin(Xc)] - BETA[numpy.argmin(Xc)]), abs(Xc[numpy.argmax(Xc)] + BETA[numpy.argmax(Xc)]))))
        
        xmin, xmax = min(X), max(abs(X))
        
        if (y_pred_beta.size != 0) and (y_pred_out_beta.size != 0) :
            ymax = max(max(y), max(y_pred_beta), max(y_pred_out_beta))
        elif (y_pred_beta.size != 0):
            ymax = max(max(y), max(y_pred_beta))
        else:
            ymax = max(max(y), max(y_pred_out_beta))
        
        ymin, ymax = 0, numpy.round(numpy.ceil(ymax))
        return xmin, xmax, ymin, ymax

def visualize_PM_reg_gn(X=None, y=None, w=None, b=None, xmin=None, xmax=None, ymin=None, ymax=None, ylabel=None, situation=None, jeu=None, lang='fr'):
    assert lang == 'fr' or lang == 'en'
    
    y_pred = get_regression_value(X, w, b)
    
    x_std = (numpy.linspace(xmin,xmax,5)[1] - numpy.linspace(xmin,xmax,5)[0]) / 10
    y_std = (numpy.linspace(ymin,ymax,5)[1] - numpy.linspace(ymin,ymax,5)[0]) / 10
    
    # figure
    if jeu == 'Apprentissage' :
        jeu_label = 'Jeu d\'entraînement' if lang == 'fr' else 'Training set'
    else :
        jeu_label = 'Jeu de test' if lang == 'fr' else 'Testing set'
        
    regression_line_label = 'Droite de régression GLM' if lang == 'fr' else 'GLM fit'
    
    pyplot.figure(figsize=(3.5,2), dpi=200)  #pyplot.figure(figsize=(4.5,3), dpi=200)
    pyplot.plot(X, y, marker='o', markersize=3.0, color='blue', linestyle='', label=jeu_label)
    idx = numpy.argsort(X.reshape(-1),  axis=0)
    pyplot.plot(X[idx], y_pred[idx], color='gold', linestyle='-', label=regression_line_label + r' $(\mathbf{v}, b)$')
    
    ax = pyplot.gca()
    x_locator = FixedLocator([int(value) if numpy.floor(value)==value else float(numpy.round(value,4)) for value in numpy.linspace(xmin,xmax,12)])
    y_locator = FixedLocator([int(value) if numpy.floor(value)==value else float(numpy.round(value,2)) for value in numpy.arange(ymin,ymax+1,int(ymax/5))])

    date = {0:'Jan.', 1:'Feb.', 2:'Mar.', 3:'Apr.', 4:'May.', 5:'Jun.', 6:'Jul.', 7:'Aug.', 8:'Sep.', 9:'Oct.', 10:'Nov.', 11:'Dec.'}
    x_formatter = FixedFormatter([date[key] for key in numpy.arange(0,12,1)])
    y_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(float(numpy.round(value,2))) for value in numpy.arange(ymin,ymax+1,int(ymax/5))])

    ax.xaxis.set_major_locator(x_locator)
    ax.yaxis.set_major_locator(y_locator)
    ax.xaxis.set_major_formatter(x_formatter)
    ax.yaxis.set_major_formatter(y_formatter)
    
    pyplot.xlabel('Date')
    pyplot.ylabel(ylabel)
    pyplot.xlim(xmin-x_std, xmax+x_std)
    pyplot.ylim(ymin-y_std, ymax+y_std)
    pyplot.legend(loc ="upper right")
    
    pyplot.savefig(saved_img_path+'/img_visualize_GLM_'+situation+'_'+jeu+'.png', dpi=500, bbox_inches='tight', transparent=True)

#### Visualization of the MoTLGLM fit (for our model)

In [ ]:
def visualize_MoTLGLM_reg_gn(X=None, y=None, Xc=None, Yc=None, Mat_dists=None, V=None, v_ext=None, B=None, b_ext=None, BETA=None, xmin=None, xmax=None, ymin=None, ymax=None, ylabel=None, situation=None, jeu=None, lang='fr'):
    assert lang == 'fr' or lang == 'en'
    
    Xc = Xc.reshape(Xc.shape[0])
    idx = numpy.argsort(Xc)
    Xc, Yc, V, B, BETA = Xc[idx], Yc[idx], V[idx], B[idx], BETA[idx]
    Xc = Xc.reshape(Xc.shape[0], 1)
    Mat_dists = Mat_dists[idx]

    # figure
    if jeu == 'Apprentissage' :
        jeu_label = 'Jeu d\'entraînement' if lang == 'fr' else 'Training set'
    else :
        jeu_label = 'Jeu de test' if lang == 'fr' else 'Testing set'
        
    locality_label = 'Localités' if lang == 'fr' else 'Regions'
    regression_line_label = 'Ajustement par régression' if lang == 'fr' else 'Regression fit'
    
    pyplot.figure(figsize=(3.5,2), dpi=200) #pyplot.figure(figsize=(4.5,3), dpi=200)
    pyplot.plot(X, y, marker='o', markersize=3.0, color='blue', linestyle='')
    pyplot.plot(Xc, Yc, marker='*', markersize=4.75, color='green', linestyle='', label=r'$\mathbf{c}_{i} = \mathbf{c}_{i_{0}}$')

    x_std = (numpy.linspace(xmin,xmax,5)[1] - numpy.linspace(xmin,xmax,5)[0]) / 10
    y_std = (numpy.linspace(ymin,ymax,5)[1] - numpy.linspace(ymin,ymax,5)[0]) / 10

    for i in range(BETA.size):
        # Plot inner region
        x_v_1 = (Yc[i] - (y_std))/(ymax - ymin + 2 * y_std) + (1 - ((ymax + y_std)/(ymax - ymin + 2 * y_std)))
        x_v_2 = (Yc[i] + (y_std))/(ymax - ymin + 2 * y_std) + (1 - ((ymax + y_std)/(ymax - ymin + 2 * y_std)))

        x_h_1 = (Xc[i] - BETA[i])/(xmax - xmin + 2 * x_std) + (1 - ((xmax + x_std)/(xmax - xmin + 2 * x_std)))
        x_h_2 = (Xc[i] + BETA[i])/(xmax - xmin + 2 * x_std) + (1 - ((xmax + x_std)/(xmax - xmin + 2 * x_std)))

        pyplot.axhline(y = Yc[i], xmin=x_h_1, xmax=x_h_2, color = 'purple')
        pyplot.axvline(x = Xc[i] - BETA[i], ymin=x_v_1, ymax=x_v_2, color = 'purple') if i != 0 else pyplot.axvline(x = Xc[i] - BETA[i], ymin=x_v_1, ymax=x_v_2, color = 'purple', label=locality_label + r' $\beta_{i} = k_{i}/\tau_{i}$')
        pyplot.axvline(x = Xc[i] + BETA[i], ymin=x_v_1, ymax=x_v_2, color = 'purple')
    
    # for V = v_1 to v_n
    for i in range(BETA.size):
        xpoints_beta, y_beta = X[Mat_dists[i] <= BETA[i]], y[Mat_dists[i] <= BETA[i]]   
        if xpoints_beta.size != 0:
            idx = numpy.argsort(xpoints_beta.reshape(-1),  axis=0)
            y_pred_beta = get_regression_value(xpoints_beta, V[i], B[i])
            pyplot.plot(xpoints_beta[idx], y_pred_beta[idx], color='gold', linestyle='-') if i != 0 else pyplot.plot(xpoints_beta[idx], y_pred_beta[idx], color='gold', linestyle='-', label=regression_line_label + r' $(\mathbf{v}_{i}, b_{i})=(\mathbf{w}_{i}, \mu_{i})$')
              
    # for v_ext = w_ext (out of beta region)
    mask = Mat_dists > BETA.reshape(BETA.size,1)
    mask = numpy.prod(mask, axis=0).astype(bool)
    xpoints_out_beta, y_out_beta = X[mask], y[mask]
    if xpoints_out_beta.size != 0:
        y_pred_out_beta = get_regression_value(xpoints_out_beta, v_ext, b_ext)
        xpoints_out_beta = xpoints_out_beta.reshape(-1)
        
        cond_low = xpoints_out_beta < (Xc[numpy.argmin(Xc)] - BETA[numpy.argmin(Xc)])
        cond_high = xpoints_out_beta > (Xc[numpy.argmax(Xc)] + BETA[numpy.argmax(Xc)])

        plot_label = False
        if cond_low.sum() != 0 or cond_high.sum() != 0:
            plot_label = True
            xpoints_out_beta_cond_low, y_pred_out_beta_cond_low = xpoints_out_beta[cond_low], y_pred_out_beta[cond_low]
            xpoints_out_beta_cond_high, y_pred_out_beta_cond_high = xpoints_out_beta[cond_high], y_pred_out_beta[cond_high]

            idx_l = numpy.argsort(xpoints_out_beta_cond_low.reshape(-1),  axis=0)
            idx_h  = numpy.argsort(xpoints_out_beta_cond_high.reshape(-1),  axis=0)
            pyplot.plot(xpoints_out_beta_cond_low[idx_l], y_pred_out_beta_cond_low[idx_l], color='indigo', linestyle='-', label=regression_line_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')
            pyplot.plot(xpoints_out_beta_cond_high[idx_h], y_pred_out_beta_cond_high[idx_h], color='indigo', linestyle='-')

        for i in range(Xc.shape[0]-1):
            cond_between = (xpoints_out_beta > (Xc[i] + BETA[i])) & (xpoints_out_beta < (Xc[i+1] - BETA[i+1]))
            xpoints_out_beta_cond_between, y_pred_out_beta_cond_between = xpoints_out_beta[cond_between], y_pred_out_beta[cond_between]
            idx_c = numpy.argsort(xpoints_out_beta_cond_between.reshape(-1),  axis=0)
            if plot_label == False and i == 0 :
                pyplot.plot(xpoints_out_beta_cond_between[idx_c], y_pred_out_beta_cond_between[idx_c], color='indigo', linestyle='-', label=regression_line_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')
            else:
                pyplot.plot(xpoints_out_beta_cond_between[idx_c], y_pred_out_beta_cond_between[idx_c], color='indigo', linestyle='-')

    pyplot.plot(Xc, Yc, marker='*', markersize=4.75, color='green', linestyle='')
    
    ax = pyplot.gca()
    x_locator = FixedLocator([int(value) if numpy.floor(value)==value else float(numpy.round(value,4)) for value in numpy.linspace(xmin,xmax,12)])
    y_locator = FixedLocator([int(value) if numpy.floor(value)==value else float(numpy.round(value,2)) for value in numpy.arange(ymin,ymax+1,int(ymax/5))])

    date = {0:'Jan.', 1:'Feb.', 2:'Mar.', 3:'Apr.', 4:'May.', 5:'Jun.', 6:'Jul.', 7:'Aug.', 8:'Sep.', 9:'Oct.', 10:'Nov.', 11:'Dec.'}
    x_formatter = FixedFormatter([date[key] for key in numpy.arange(0,12,1)])
    y_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(float(numpy.round(value,2))) for value in numpy.arange(ymin,ymax+1,int(ymax/5))])

    ax.xaxis.set_major_locator(x_locator)
    ax.yaxis.set_major_locator(y_locator)
    ax.xaxis.set_major_formatter(x_formatter)
    ax.yaxis.set_major_formatter(y_formatter)
    
    pyplot.xlabel('Date')
    pyplot.ylabel(ylabel)
    pyplot.xlim(xmin-x_std, xmax+x_std)
    pyplot.ylim(ymin-y_std, ymax+y_std)
    pyplot.grid(linestyle = 'dashed', linewidth = 1)
    #pyplot.legend(loc ="upper left", bbox_to_anchor=(0.5, 0.49)) ##################

    pyplot.savefig(saved_img_path+'/img_visualize_MoTLGLM_'+situation+'_'+jeu+'_search.png', dpi=500, bbox_inches='tight', transparent=True)

    figsize = (1,6)
    pyplot.figure(figsize=figsize, dpi=200)
    ax_leg = pyplot.gca()
    ax_leg.legend(*ax.get_legend_handles_labels(), ncol=2, loc='center', fontsize=9, facecolor='white', edgecolor='black', framealpha=0.05)
    ax_leg.axis('off')
    pyplot.savefig(saved_img_path+'/img_MoTLGLM_legend_search.png', dpi=1000, bbox_inches='tight', transparent=True)      